In [ ]:
!nvidia-smi

Mon Jun 14 04:56:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    38W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install tensorflow-gpu==2.5.0
#!pip install tensorflow==2.5.0
!pip install ktrain==0.26.3
#!pip list

     |████████████████████████████████| 25.3MB 114kB/s 
     |████████████████████████████████| 6.8MB 29.4MB/s 
     |████████████████████████████████| 983kB 17.6MB/s 
     |████████████████████████████████| 266kB 30.9MB/s 
     |████████████████████████████████| 1.9MB 28.7MB/s 
     |████████████████████████████████| 1.2MB 27.0MB/s 
     |████████████████████████████████| 471kB 30.9MB/s 
     |████████████████████████████████| 3.3MB 31.0MB/s 
     |████████████████████████████████| 901kB 31.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.26.3-cp37-none-any.whl size=25282390 sha256=a14837ddde91df7bf23caa2d594228d6f38abcb7e240b04f7c717db6a225270c
  Stored in directory: /root/.cache/pip/wheels/16/05/be/d6e659b3349016b1059e19fa028f165af4eeae2c196f329112
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=fc6cfda0339f00cbab3f8c676bfc3e52969bbfbea028b9f8a73f81180af8d4e4
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7d

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text 
import tensorflow as tf

In [ ]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)

In [ ]:
data_train.sample(5)

,Reviews,Sentiment
14140,I can only assume that the other reviewers of ...,neg
7978,"Once again, Disney manages to make a children'...",neg
13619,"In what is a truly diverse cast, this show hit...",pos
24819,This 1939 film from director John Ford and wri...,pos
14819,"As a great admirer of Marlene Dietrich, I had ...",neg


In [ ]:
#Check what the classification model mare there
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [ ]:
(train, val, preproc) = text.texts_from_df(train_df=data_train,text_column="Reviews",label_columns="Sentiment",
                   val_df=data_test,maxlen=400,preprocess_mode='distilbert') #bert uses 512 word

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0



preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


In [ ]:
model = text.text_classifier(name='distilbert',train_data=train,preproc=preproc)

Is Multi-Label? False
maxlen is 400



done.


In [ ]:
learner = ktrain.get_learner(model=model,train_data=train,val_data=val,batch_size=6)

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
4167/4167 [==============================] - 3068s 732ms/step - loss: 0.2923 - accuracy: 0.8791 - val_loss: 0.1586 - val_accuracy: 0.9434
Epoch 2/5
4167/4167 [==============================] - 3052s 731ms/step - loss: 0.1888 - accuracy: 0.9293 - val_loss: 0.1012 - val_accuracy: 0.9715
Epoch 3/5
4167/4167 [==============================] - 3050s 730ms/step - loss: 0.1270 - accuracy: 0.9549 - val_loss: 0.0437 - val_accuracy: 0.9870
Epoch 4/5
4167/4167 [==============================] - 3056s 732ms/step - loss: 0.0576 - accuracy: 0.9815 - val_loss: 0.0145 - val_accuracy: 0.9970
Epoch 5/5
  68/4167 [..............................] - ETA: 38:36 - loss: 0.0428 - accuracy: 0.9853

KeyboardInterrupt: ignored

In [ ]:
predictor = ktrain.get_predictor(learner.model,preproc)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
predictor.save('/content/distilberttf')

In [ ]:
!zip -r /content/drive/MyDrive/NLP_models/distilberttf.zip /content/drive/MyDrive/NLP_models/distilberttf

  adding: content/drive/MyDrive/NLP_models/distilberttf/ (stored 0%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/config.json (deflated 43%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/tf_model.h5 (deflated 8%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/tokenizer_config.json (deflated 36%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/special_tokens_map.json (deflated 40%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/vocab.txt (deflated 53%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/tf_model.preproc (deflated 44%)
  adding: content/drive/MyDrive/NLP_models/distilberttf/requirement.txt (deflated 54%)


In [ ]:
!pip freeze > /content/drive/MyDrive/NLP_models/distilberttf/requirement.txt

In [ ]:
!python --version

Python 3.7.10


In [ ]:
predictor=ktrain.load_predictor('/content/distilberttf')

In [ ]:
data = ["Excellent direction from Sudha . Cinematography was superb. Please watch with your family . A good motivational film",
        "i will not not see this movie again it was so bag", "actor surya acting is good"]

In [ ]:
predictor.predict(data)

['pos', 'neg', 'pos']

In [ ]:
predictor.get_classes()

['neg', 'pos']

In [ ]:
predictor.predict(data,return_proba=True)

array([[0.0019092 , 0.9980908 ],
       [0.99345064, 0.00654933],
       [0.09375734, 0.90624267]], dtype=float32)

--2021-06-13 14:18:11--  https://drive.google.com/file/d/13fUY2V5wBCFLu8KuFIZIi8_HLNspU5Kt/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 108.177.15.101, 108.177.15.113, 108.177.15.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.15.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing.1’

view?usp=sharing.1      [ <=>                ]  64.73K  --.-KB/s    in 0.07s   

2021-06-13 14:18:11 (910 KB/s) - ‘view?usp=sharing.1’ saved [66281]

